# Class4-Data_Visualization-Week20

__Data:__ https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

## We want to answer these questions by visualizations (I used many methods here. You can check them below.)

* Global Suicides(per 100K)-trend over time 1985-2016
* Global Suicides(per 100K) by Continent
* Global Suicides(per 100k) by Gender and trend over time 1985-2016
* Population-gdp_per_capita Plot
* Correlation between GDP(per Capita) and suicides per 100k
* Generation hue Gender Counter
* Which age of people suicide a most
* Which generation of people suicide a most

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/suicide-rates-overview-1985-to-2016/master.csv")
df.sample(10)

In [ ]:
df.describe(include='all').T

In [ ]:
df.suicides_no.hist()

Rename the columns.

In [ ]:
df.rename(columns={"suicides/100k pop":"suicides_pop","HDI for year":"HDI_for_year",
                  " gdp_for_year ($) ":"gdp_for_year"," gdp_per_capita ($) ":"gdp_per_capita",}, inplace=True)
print(df.columns)


Change the comma in gdp to nothing to calculate it further easily. And to sort the age arrange the years.

In [ ]:
df["gdp_for_year"] = df["gdp_for_year"].str.replace(",","").astype(np.int64)
df["age"] = df["age"].str.replace("5-14 years","05-14 years")

First of all, we can check the suicide rate over time globally. 

In [ ]:
df_year = pd.DataFrame(df.groupby("year")["suicides_no"].sum())
df_year.reset_index(inplace=True)
sns.lineplot(x= df_year['year'], y=df_year['suicides_no'], ci = None)
plt.legend('suicide over year')
plt.show()

As we can see, generally the number suicide is arround 15000 to 25000 between the years 1992-2015

In [ ]:
df_suicide = df.copy()
df_suicide

In [ ]:
df_country = pd.read_csv("/kaggle/input/country-to-continent/countryContinent.csv")
df_country

In [ ]:
def find_mismatch (array_1, array_2, only_out=True):
    for i in array_1.unique():
         if i in array_2.unique():
            pass
         else:
            print("Not have :",i)
            
    
find_mismatch(df.country, df_country.country, True)

In [ ]:
df_country.loc[df_country.country.str.contains("Maca"),'country'] = 'Macau'
df_country.loc[118,'country'] = 'Republic of Korea'
df_country.loc[df_country.country.str.contains("Vincent"),'country'] = 'Saint Vincent and Grenadines'
df_country.loc[df_country.country.str.contains("King"),'country'] = 'United Kingdom'
df_country.loc[df_country.country.str.contains("United States of America"),'country'] = 'United States'

find_mismatch(df.country, df_country.country, True)

In [ ]:
df = df.merge(df_country[['country', 'continent', 'code_3']])
df.info()

In [ ]:
df_cont = pd.DataFrame(df.groupby("continent")['suicides_no'].sum()).reset_index()
df_cont

In [ ]:
sns.lineplot(x= df_cont['continent'], y=df_cont['suicides_no'])
plt.legend('suicide per continent')
plt.show()

 As we can see, most suicedes has been commited in Europe Asia and America. Now we can check the suicide rate of men and women seperately to see which gender is more aimed to suicide/. 

In [ ]:
df_men = df[df.sex == "male"]
df_women = df[df.sex == "female"]
sns.lineplot(x= df_men.year, y=df.suicides_no, ci = None)
sns.lineplot(x=df_women.year, y=df.suicides_no, ci = None)
plt.legend(["male", 'female'])
plt.show()

As we can see in the graph, males are more tend to suicide. There could be lots of reasons. Let's check the age.

In [ ]:
idx = ["suicides_no", "population"]
df_age = df.groupby(["year","age"])[idx].sum()
df_reset = df_age.copy().reset_index()
plt.figure(figsize=(9,6))
sns.lineplot(x="year", y=df_reset.suicides_no*100/df_reset.population, hue = "age",
             data = df_reset, linewidth = 2.5, style = "age", markers=True
            , dashes=False)
plt.xticks(rotation = 90)
plt.show()

As seen above graph, 75+ years are most tend to suicide.

# Population-gdp_per_capita Plot

In [ ]:
df

In [ ]:
df2= df.groupby(["country", "year"]).agg({"population":'sum', 'suicides_no':'sum', "gdp_per_capita":'mean'}).reset_index()

for i in df2['country'].unique():
    plt.subplots(1,1)
    sns.barplot(x="population", y="gdp_per_capita", data=df2[df2['country'] == i])
    plt.xlabel("Population", fontsize=18)
    plt.ylabel("GDP Per Capita", fontsize=18)
    plt.title("Population and GDP Per Capita in " + i, fontsize=15)
    display(plt.show())

# Correlation between GDP(per Capita) and suicides per 100k

In [ ]:
corr = df.corr(method='spearman')
plt.figure(figsize=(15,15))
sns.heatmap(corr, vmax=.8, linewidths=0.01, square=True, annot=True, cmap='RdBu', linecolor='black')

In [ ]:
plt.plot("suicides_pop", 'gdp_per_capita', data=df, linestyle='', marker='o', markersize=0.5)
plt.xlabel("Suicide Per 100K Population")
plt.ylabel("GDP Per Capita")
plt.title("Suicide Rate vs GDP")
plt.show()

In [ ]:
sns.regplot(x=df.suicides_pop, y=df.gdp_per_capita, color='b')
stats.spearmanr(df.gdp_per_capita, df.suicides_pop)

# Generation hue Gender Counter


In [ ]:
df_generation = df.groupby(['generation', 'year'])['suicides_no'].sum().reset_index()
df_generation.sample(10)

In [ ]:
plt.figure(figsize=(9,6))
sns.lineplot(x="year", y=df_generation.suicides_no*100/df_reset.population, hue = "generation",
             data = df_generation, linewidth = 2.5, style = "generation", markers=True
            , dashes=False)
plt.xticks(rotation = 90)
plt.show()

As seen above graph, 75+ aged people are more tend to suicide. There might be many reason that is happening.

If we check the last table 'Boomers' Generation are most tend to suicide through out the years. 